In [1]:
from os import environ
environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.4.1  --driver-memory 10g  --executor-memory 5g pyspark-shell'

import pyspark
sc = pyspark.SparkContext(appName="StackOverflow")

from pyspark.sql import SQLContext,SparkSession
from pyspark.sql.functions import col,explode, udf
sqlContext = SQLContext(sc)
sqlContext.setConf("spark.sql.shuffle.partitions", "300")
sqlContext.setConf("spark.memory.offHeap.size","10g")

spark = SparkSession.builder \
    .appName('DataFrame_2') \
    .master('local[*]') \
    .getOrCreate()

In [2]:
from datetime import datetime, timedelta
dt_format = "%Y-%m-%dT%H:%M:%S.%f"

rawBadgeData = sc.textFile("/home/adam/bigdata/dane/codereview/Badges.xml")
rawCommentData = sc.textFile("/home/adam/bigdata/dane/codereview/Comments.xml")
rawVoteData = sc.textFile("/home/adam/bigdata/dane/codereview/Votes.xml")
rawUserData = sc.textFile("/home/adam/bigdata/dane/codereview/Users.xml")
rawPostData = sc.textFile("/home/adam/bigdata/dane/codereview/Posts.xml")
#rawBadgeData.take(5)
#rawCommentData.take(5)
#rawVoteData.take(5)
rawUserData.take(5)
#rawPostData.take(20)

['<?xml version="1.0" encoding="utf-8"?>',
 '<users>',
 '  <row Id="-1" Reputation="1" CreationDate="2011-01-19T19:31:34.353" DisplayName="Community" LastAccessDate="2011-01-19T19:31:34.353" WebsiteUrl="http://meta.stackexchange.com/" Location="on the server farm" AboutMe="&lt;p&gt;Hi, I\'m not really a person.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;I\'m a background process that helps keep this site clean!&lt;/p&gt;&#xA;&#xA;&lt;p&gt;I do things like&lt;/p&gt;&#xA;&#xA;&lt;ul&gt;&#xA;&lt;li&gt;Randomly poke old unanswered questions every hour so they get some attention&lt;/li&gt;&#xA;&lt;li&gt;Own community questions and answers so nobody gets unnecessary reputation from them&lt;/li&gt;&#xA;&lt;li&gt;Own downvotes on spam/evil posts that get permanently deleted&lt;/li&gt;&#xA;&lt;li&gt;Own suggested edits from anonymous users&lt;/li&gt;&#xA;&lt;li&gt;&lt;a href=&quot;http://meta.stackexchange.com/a/92006&quot;&gt;Remove abandoned questions&lt;/a&gt;&lt;/li&gt;&#xA;&lt;/ul&gt;&#xA;" Views="0" UpV

In [3]:
from itertools import zip_longest
transData = rawUserData.filter(lambda x: x.startswith('  <row')) \
                   .map(lambda row: [x for x in row.split('"')  if '=' not in x[-1] and '>' not in x[-1]])
transData2 = rawUserData.filter(lambda x: x.startswith('  <row')) \
                   .map(lambda row: [x.replace('=','') for x in row.split('"')  if '=' in x[-1] and '>' not in x[-1]])
    
transData3 = rawUserData.filter(lambda x: x.startswith('  <row')) \
                   .map(lambda row: [x.strip().replace('=','').replace('<row','').replace('/>','')  for x in row.split('"') ])  \
                   .map(lambda elem: dict(zip(elem[::2], elem[1::2])))
                   
                  
                   
# transData.take(5)  
# transData3.take(5)  

# from pyspark.sql import SparkSession
# from pyspark.sql import Row
# from collections import OrderedDict

# #spark.createDataFrame(df_badge).show(truncate=False)
# #df_badge.map(lambda l: Row(**dict(l))).toDF().show(truncate=False)
# df_u = transData3.map(lambda l: Row(**OrderedDict(l)))

from pyspark.sql.types import TimestampType,IntegerType,StringType,StructType,StructField



def readXMLs(rawData):
    
    transData = rawData.filter(lambda x: x.startswith('  <row')) \
                       .map(lambda row: [x.replace('=','').replace('<row','').replace('/>','').strip()  for x in row.split('"') ])  \
                       .map( lambda elem: dict(zip(elem[::2], elem[1::2])))
    df = spark.createDataFrame(transData)
      
    return df

df_posts = readXMLs(rawPostData)
df_users = readXMLs(rawUserData)
df_badges = readXMLs(rawBadgeData)
df_votes = readXMLs(rawVoteData)
df_comments = readXMLs(rawCommentData)

df_users.show()


import pyspark.sql.functions as f




df_p = df_posts.withColumn('Id', df_posts.Id.cast(IntegerType())) \
                  .withColumn('PostTypeId', df_posts.PostTypeId.cast(IntegerType())) \
                  .withColumn('AcceptedAnswerId', df_posts.AcceptedAnswerId.cast(IntegerType())) \
                  .withColumn('CreationDate', df_posts.CreationDate.cast(TimestampType())) \
                  .withColumn('Score', df_posts.Score.cast(IntegerType())) \
                  .withColumn('ViewCount', df_posts.ViewCount.cast(IntegerType())) \
                  .withColumn('Body', df_posts.Body.cast(StringType())) \
                  .withColumn('OwnerUserId', df_posts.OwnerUserId.cast(IntegerType())) \
                  .withColumn('LastEditorUserId', df_posts.LastEditorUserId.cast(IntegerType())) \
                  .withColumn('LastEditDate', df_posts.LastEditDate.cast(TimestampType())) \
                  .withColumn('LastActivityDate', df_posts.LastActivityDate.cast(TimestampType())) \
                  .withColumn('Title', df_posts.Title.cast(StringType())) \
                  .withColumn('Tags', df_posts.Tags.cast(StringType())) \
                  .withColumn('AnswerCount', df_posts.AnswerCount.cast(IntegerType())) \
                  .withColumn('CommentCount', df_posts.CommentCount.cast(IntegerType())) \
                  .select('Id','PostTypeId','AcceptedAnswerId','CreationDate','Score','ViewCount','Body','OwnerUserId','LastActivityDate','Title','Tags','AnswerCount','CommentCount') 

df_p_a = df_posts.where(df_posts.PostTypeId == '2').withColumn('Id', df_posts.Id.cast(IntegerType())) \
                  .withColumn('PostTypeId', df_posts.PostTypeId.cast(IntegerType())) \
                  .withColumn('AcceptedAnswerId', df_posts.AcceptedAnswerId.cast(IntegerType())) \
                  .withColumn('AnswerCreationDate', df_posts.CreationDate.cast(TimestampType())) \
                  .withColumn('Score', df_posts.Score.cast(IntegerType())) \
                  .withColumn('ViewCount', df_posts.ViewCount.cast(IntegerType())) \
                  .withColumn('Body', df_posts.Body.cast(StringType())) \
                  .withColumn('OwnerUserId', df_posts.OwnerUserId.cast(IntegerType())) \
                  .withColumn('LastEditorUserId', df_posts.LastEditorUserId.cast(IntegerType())) \
                  .withColumn('LastEditDate', df_posts.LastEditDate.cast(TimestampType())) \
                  .withColumn('LastActivityDate', df_posts.LastActivityDate.cast(TimestampType())) \
                  .withColumn('Title', df_posts.Title.cast(StringType())) \
                  .withColumn('Tags', df_posts.Tags.cast(StringType())) \
                  .withColumn('AnswerCount', df_posts.AnswerCount.cast(IntegerType())) \
                  .withColumn('CommentCount', df_posts.CommentCount.cast(IntegerType())) \
                  .select('Id','PostTypeId','AcceptedAnswerId','AnswerCreationDate','Score','ViewCount','Body','OwnerUserId','LastActivityDate','Title','Tags','AnswerCount','CommentCount') 
                  
df_p_a.show()
                
df_u = df_users.withColumn('OwnerUserId', df_users.Id.cast(IntegerType())) \
                  .withColumn('Reputation', df_users.Reputation.cast(IntegerType())) \
                  .withColumn('UserCreationDate', df_users.CreationDate.cast(TimestampType())) \
                  .withColumn('DisplayName', df_users.DisplayName.cast(StringType())) \
                  .withColumn('LastAccessDate', df_users.LastAccessDate.cast(TimestampType())) \
                  .withColumn('UserLocation', df_users.Location.cast(StringType())) \
                  .withColumn('UserAboutMe', df_users.AboutMe.cast(StringType())) \
                  .withColumn('UserWebsiteUrl', df_users.WebsiteUrl.cast(StringType())) \
                  .withColumn('Views', df_users.Views.cast(IntegerType())) \
                  .withColumn('UpVotes', df_users.UpVotes.cast(IntegerType())) \
                  .withColumn('DownVotes', df_users.DownVotes.cast(IntegerType())) \
                  .withColumn('AccountId', df_users.AccountId.cast(IntegerType())) \
                  .withColumn('Flag_AboutMe',f.when(df_users.AboutMe.isNotNull(),1).otherwise(0)) \
                  .withColumn('Flag_WebsiteUrl',f.when(df_users.WebsiteUrl.isNotNull(),1).otherwise(0)) \
                  .withColumn('Flag_Location',f.when(df_users.Location.isNotNull(),1).otherwise(0)) \
                  .select('OwnerUserId','Reputation','UserCreationDate','DisplayName','LastAccessDate','UserLocation','Flag_WebsiteUrl','Flag_AboutMe','Flag_Location','Views','UpVotes','DownVotes','WebsiteUrl') 
                   
                
#  .withColumn('UserProfileImageUrl', df_users.ProfileImageUrl.cast(StringType())) \
#  .withColumn('Flag_ProfileImageUrl',f.when(df_users.ProfileImageUrl.isNotNull(),1).otherwise(0)) \
                
                

df_b = df_badges.withColumn('Id', df_badges.Id.cast(IntegerType())) \
                  .withColumn('UserId', df_badges.UserId.cast(IntegerType())) \
                  .withColumn('BadgeDate', df_badges.Date.cast(TimestampType())) \
                  .withColumn('Class', df_badges.Class.cast(IntegerType())) \
                  .select('Id','UserId','Name','BadgeDate','Class') 
                  
df_c = df_comments.withColumn('CommentId', df_comments.Id.cast(IntegerType())) \
                  .withColumn('PostId', df_comments.PostId.cast(IntegerType())) \
                  .withColumn('CommentCreationDate', df_comments.CreationDate.cast(TimestampType())) \
                  .withColumn('Score', df_comments.Score.cast(IntegerType())) \
                  .withColumn('Text', df_comments.Text.cast(StringType())) \
                  .withColumn('UserId', df_comments.UserId.cast(IntegerType())) \
                  .select('CommentId','PostId','Score','Text','CommentCreationDate','UserId') 
                
df_v = df_votes.withColumn('VoteId', df_votes.Id.cast(IntegerType())) \
                  .withColumn('PostId', df_votes.PostId.cast(IntegerType())) \
                  .withColumn('VoteCreationDate', df_votes.CreationDate.cast(TimestampType())) \
                  .withColumn('VoteTypeId', df_votes.VoteTypeId.cast(IntegerType())) \
                  .select('VoteId','PostId','VoteTypeId','VoteCreationDate')                 
                
                    
# df_u = df_users.withColumn('OwnerUserId', df_users.Id.cast(IntegerType())) \
#                   .withColumn('Reputation', df_users.Reputation.cast(IntegerType())) \
#                   .withColumn('UserCreationDate', df_users.CreationDate.cast(TimestampType())) \
#                   .withColumn('DisplayName', df_users.DisplayName.cast(StringType())) \
#                   .withColumn('LastAccessDate', df_users.LastAccessDate.cast(TimestampType())) \
#                   .withColumn('Location', df_users.Location.cast(StringType())) \
#                   .withColumn('AboutMe', df_users.AboutMe.cast(StringType())) \
#                   .withColumn('Views', df_users.Views.cast(IntegerType())) \
#                   .withColumn('UpVotes', df_users.UpVotes.cast(IntegerType())) \
#                   .withColumn('DownVotes', df_users.DownVotes.cast(IntegerType())) \
#                   .withColumn('AccountId', df_users.AccountId.cast(IntegerType())) \
#                   .withColumn('FlagAboutMe',f.when(df_users.AboutMe.isNotNull(),1).otherwise(0)) \
#                   .withColumn('FlagLocation',f.when(df_users.Location.isNotNull(),1).otherwise(0)) \
#                   .select('OwnerUserId','Reputation','UserCreationDate','DisplayName','LastAccessDate','Location', 'Views','UpVotes','DownVotes','FlagWebsiteUrl','FlagAboutMe','FlagLocation') 


                    
                    
#df_u.show(truncate = False)

# type(dtaDF)

#df_b1.printSchema()


#type(transData2)
# from pyspark.sql import SparkSession
# from pyspark.sql import Row

#transData3.map(lambda l: Row(**dict(l))).take(5)

#dict(zip(transData3[::2], transData3[1::2])).take(5)

#spark.createDataFrame(df_badge).show(truncate=False)
#df_badge.map(lambda l: Row(**dict(l))).toDF().show(truncate=False)
# df_b = df_badge.map(lambda l: Row(**dict(l)))

# data = [dict(zip(transData, transData2)) for entries in transData]
# print(data)
               

/home/adam/anaconda3/lib/python3.6/site-packages/pyspark/sql/session.py:360: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


+--------------------+---------+--------------------+--------------+---------+---+--------------------+------------------+----------+-------+-----+--------------------+
|             AboutMe|AccountId|        CreationDate|   DisplayName|DownVotes| Id|      LastAccessDate|          Location|Reputation|UpVotes|Views|          WebsiteUrl|
+--------------------+---------+--------------------+--------------+---------+---+--------------------+------------------+----------+-------+-----+--------------------+
|&lt;p&gt;Hi, I'm ...|       -1|2011-01-19T19:31:...|     Community|     4676| -1|2011-01-19T19:31:...|on the server farm|         1|   5606|    0|http://meta.stack...|
|&lt;p&gt;Develope...|        2|2011-01-19T20:26:...|  Geoff Dalgas|        0|  2|2017-12-05T22:42:...|     Corvallis, OR|       101|      0|   37|http://stackoverf...|
|&lt;p&gt;co-found...|     1998|2011-01-19T20:50:...|        Emmett|        0|  3|2015-11-25T21:04:...| San Francisco, CA|       101|      0|   23|http://m

In [4]:
from pyspark.sql import Window
from pyspark.sql.types import TimestampType,LongType
import pycountry
from pycountry_convert import (
    map_countries,
    country_alpha2_to_country_name,
    country_name_to_country_alpha2,
    country_alpha3_to_country_alpha2,
    country_alpha2_to_continent_code,
    COUNTRY_NAME_FORMAT_DEFAULT
)

import pyspark.sql.functions as f

def get_Country(text):
    if text is not None and text.strip() !='Antarctica':
        if ','  in  text and text.count(',')==1 and len(text.split(',')[1].lstrip())!=2:
            country = text.split(',')[1].lstrip()
        elif ','  in  text and text.count(',')==2 and len(text.split(',')[2].lstrip())!=2:
            country = text.split(',')[2].lstrip()  
        elif ','  in  text and len(text.split(',')[1].lstrip())==2 and text.split(',')[1].lstrip() != 'UK':
            country = 'United States of America'     
        else:
             
            if 'Texas' in  text:
                country = 'United States of America'
            elif 'Florida' in  text:  
                country = 'United States of America'
            elif 'Pennsylvania' in  text:  
                country = 'United States of America'
            elif 'Minnesota' in  text:  
                country = 'United States of America'
            elif 'USA' in  text:  
                country = 'United States of America'
            elif 'New Jersey' in  text:  
                country = 'United States of America'
            elif 'NYC' in  text:  
                country = 'United States of America'
            elif 'Ohio' in  text:  
                country = 'United States of America'
            elif 'Utah' in  text:  
                country = 'United States of America'
            elif 'Michigan' in  text:  
                country = 'United States of America'
            elif 'Colorado' in  text:  
                country = 'United States of America'
            elif 'Arkansas' in  text:  
                country = 'United States of America'
            elif 'Arizona' in  text:  
                country = 'United States of America'
            elif 'Illinois' in  text:  
                country = 'United States of America'
            elif 'Indiana' in  text:  
                country = 'United States of America'
            elif 'Chicago' in  text:  
                country = 'United States of America'
            elif 'Tennessee' in  text:  
                country = 'United States of America'
            elif 'North Carolina' in  text:  
                country = 'United States of America'
            elif 'South Carolina' in  text:  
                country = 'United States of America'
            elif 'Missouri' in  text:  
                country = 'United States of America'
            elif 'Connecticut' in  text:  
                country = 'United States of America'
            elif 'Indiana' in  text:  
                country = 'United States of America'
            elif 'Deutschland' in  text:  
                country = 'Germany'
            elif 'Barcelona' in  text:  
                country = 'Spain'
            elif 'Beijing' in  text:  
                country = 'China'
            elif 'California' in  text:  
                country = 'United States of America'
            elif 'New York' in  text:  
                country = 'United States of America'
            elif 'Wisconsin' in  text:  
                country = 'United States of America'
            elif 'Nebraska' in  text:  
                country = 'United States of America'
            elif 'Northern Virginia' in  text:  
                country = 'United States of America'
            elif 'Oregon' in  text:  
                country = 'United States of America'
            elif 'Missouri' in  text:  
                country = 'United States of America'
            elif 'Iowa' in  text:  
                country = 'United States of America'
            elif 'Alabama' in  text:  
                country = 'United States of America'
            elif 'Syracuse' in  text:  
                country = 'United States of America'
            elif 'Phoenix' in  text:  
                country = 'United States of America'
            elif 'Wyoming' in  text:  
                country = 'United States of America'
            elif 'Los Angeles' in  text:  
                country = 'United States of America'
            elif 'Dallas' in  text:  
                country = 'United States of America'
            elif 'San Francisco' in  text:  
                country = 'United States of America'
            elif 'San Diego' in  text:  
                country = 'United States of America'
            elif 'Seattle' in  text:  
                country = 'United States of America'
            elif 'New Delhi' in  text:  
                country = 'India'
            elif 'Chennai' in  text:  
                country = 'India'
            elif 'Mumbai' in  text:  
                country = 'India'
            elif 'Nagpur' in  text:  
                country = 'India'
            elif 'Ernakulam' in  text:  
                country = 'India'
            elif 'bangalore' in  text:  
                country = 'India'
            elif 'Bangalore' in  text:  
                country = 'India'
            elif 'London' in  text:  
                country = 'United Kingdom'
            elif 'Leeds' in  text:  
                country = 'United Kingdom'
            elif 'Liverpool' in  text:  
                country = 'United Kingdom'
            elif 'Yorkshire' in  text:  
                country = 'United Kingdom'
            elif 'Manchester' in  text:  
                country = 'United Kingdom'
            elif 'UK' in  text:  
                country = 'United Kingdom'
            elif 'Brittany' in  text:  
                country = 'United Kingdom'
            elif 'Ottawa' in  text:  
                country = 'Canada'
            elif 'Quebec' in  text:  
                country = 'Canada'
            elif 'Ontario' in  text:  
                country = 'Canada'
            elif 'Toronto' in  text:  
                country = 'Canada'
            elif 'Montreal' in  text:  
                country = 'Canada'
            elif 'Polska' in  text:  
                country = 'Poland'
            elif 'Warsaw' in  text:  
                country = 'Poland'  
            elif 'Warszawa' in  text:  
                country = 'Poland'  
            elif 'Gdynia' in  text:  
                country = 'Poland'  
            elif 'Vienna' in  text:  
                country = 'Austraia'  
            elif 'Copenhagen' in  text:  
                country = 'Denmark' 
            elif 'Berlin' in  text:  
                country = 'Germany'
            elif 'Munich' in  text:  
                country = 'Germany'
            elif 'The Netherlands' in  text:  
                country = 'The Netherland'
            elif 'Montreal' in  text:  
                country = 'Canada'
            elif 'Vancouver' in  text:  
                country = 'Canada'
            elif 'Sydney' in  text:  
                country = 'Australia'
            elif 'Aucland' in  text:  
                country = 'Australia'
            elif 'Melbourne' in  text:  
                country = 'Australia'
            elif 'Novi Sad' in  text:  
                country = 'Serbia'
            elif 'Split' in  text:  
                country = 'Croatia'
            elif 'Swerige' in  text:  
                country = 'Sweden'
            elif 'Moscow' in  text:  
                country = 'Russia'
            elif 'Minsko' in  text:  
                country = 'Belarus'
            elif 'Dnipro' in  text:  
                country = 'Ukraine'
            elif 'Odessa' in  text:  
                country = 'Ukraine'
            elif 'oslo' in  text:  
                country = 'Norway'
            elif 'paris' in  text:  
                country = 'France'
            elif 'Paris' in  text:  
                country = 'France'
            elif 'Amseterdam' in  text:  
                country = 'The Netherland'
            elif 'België' in  text:  
                country = 'Belgium'
            elif '0x0000000' in  text:  
                country = 'Brak'
            else: 
                country = text
    else:
        country = 'Brak'
    return country    




country_map = [key for key in sorted(map_countries(cn_name_format = COUNTRY_NAME_FORMAT_DEFAULT).keys())]
#print(country_map)
# def get_Continent(text):        
#     continent = None
#     if text.strip() in country_map and text.strip() !='Antarctica':
#         continent = country_alpha2_to_continent_code(country_name_to_country_alpha2(text))
#     return continent  

def get_Continent(text):        
    continent = None
    if text.strip() in country_map and text.strip() !='Antarctica':
        continent = country_alpha2_to_continent_code(country_name_to_country_alpha2(text))
    elif text.strip() not in country_map  and text.strip() !='Antarctica':   
        if 'Europe' in text:
            continent = 'EU'
        elif 'Northern Europe' in text:
            continent = 'EU'
        elif 'European Union' in text:
            continent = 'EU'
        elif 'U.S.A.' in text:
            continent = 'NA'
        elif 'United States of America.' in text:
            continent = 'NA'
        elif 'WA USA' in text:
            continent = 'NA'
        elif 'Northeast US' in text:
            continent = 'NA'
        elif 'USA' in text:
            continent = 'NA'
        elif 'NY USA' in text:
            continent = 'NA'
        elif 'Nederland' in text:
            continent = 'EU'
        elif 'The netherlands' in text:
            continent = 'EU'
        elif 'the netherlands' in text:
            continent = 'EU'
        elif 'The Kingdom of the Netherlands' in text:
            continent = 'EU'
        elif 'Polska' in text:
            continent = 'EU'
        elif 'Deutschland' in text:
            continent = 'EU'
        elif 'germany' in text:
            continent = 'EU'
        elif 'Italia' in text:
            continent = 'EU'
        elif 'Lietuva' in text:
            continent = 'EU'
        elif 'España' in text:
            continent = 'EU'
        elif 'Spanien' in text:
            continent = 'EU'
        elif 'Montréal' in text:
            continent = 'NA'
        elif 'Ontario Canada' in text:
            continent = 'NA'
        elif 'Perú' in text:
            continent = 'SA'
        elif 'England' in text:
            continent = 'EU'
        elif 'Österreich' in text:
            continent = 'EU'
        elif 'Česko' in text:
            continent = 'EU'
        elif 'România' in text:
            continent = 'EU'
        elif 'Norveç' in text:
            continent = 'EU'
        elif 'North of Norway' in text:
            continent = 'EU'
        elif 'Germnay' in text:
            continent = 'EU'
        elif 'Moscau' in text:
            continent = 'EU'
        elif 'Espagne' in text:
            continent = 'EU'
        elif 'Repubblica Ceca' in text:
            continent = 'EU'
        elif 'Украина' in text:
            continent = 'EU'
        elif 'North Cyprus' in text:
            continent = 'EU'
        elif 'Danmark' in text:
            continent = 'EU'
        elif 'Беларусь' in text:
            continent = 'EU'
        elif 'Россия' in text:
            continent = 'EU'
        elif 'Hrvatska' in text:
            continent = 'EU'
        elif 'Schweiz' in text:
            continent = 'EU'
        elif 'Szwecja' in text:
            continent = 'EU'
        elif 'Türkiye' in text:
            continent = 'EU'
        elif 'GB' in text:
            continent = 'EU'
        elif 'Uk' in text:
            continent = 'EU'
        elif 'U.K.' in text:
            continent = 'EU'
        elif 'England United Kingdom' in text:
            continent = 'EU'
        elif 'Morroco' in text:
            continent = 'AF'
        elif 'Korea' in text:
            continent = 'AS'
        elif 'Western Australia' in text:
            continent = 'OC'
        elif 'South America' in text:
            continent = 'SA'
        elif 'Dnipro' in  text:  
            continent = 'EU'
        elif 'Vienna' in  text:  
            continent = 'EU'
        elif 'Odessa' in  text:  
            continent = 'EU'
        elif 'Dubai U.A.E' in  text:  
            continent = 'AS'
        elif 'Dubai - United Arab Emirates' in  text:  
            continent = 'AS'
        else:
            continent = text
    else:
        continent = 'Brak'
            
    return continent 





def get_Alpha2(text):        
    alpha = None
    if text.strip() in country_map and text.strip() !='Antarctica':
        alpha = country_name_to_country_alpha2(text)
    return alpha 


extractCountry_udf = f.udf(lambda x: get_Country(x))
extractAlpha2_udf = f.udf(lambda x: get_Alpha2(x))
extractContinent_udf = f.udf(lambda x: get_Continent(x))


days = lambda i: i * 86400 
windowval_q = (Window.partitionBy('OwnerUserId').orderBy(col('CreationDate').cast(LongType())) 
             .rangeBetween(-days(365), 0))

windowval2_q = Window.partitionBy("OwnerUserId").orderBy("CreationDate")

windowval = (Window.partitionBy('OwnerUserId').orderBy('CreationDate')
             .rangeBetween(Window.unboundedPreceding, 0))

windowval2 = Window.partitionBy('OwnerUserId').orderBy("CreationDate")

windowval_c = Window.partitionBy("Id").orderBy("CreationDate")
windowval2_c = Window.partitionBy("UserId").orderBy("CommentCreationDate")





df_p_q_c = df_p.where(df_p.PostTypeId == 1).join(df_c.select('CommentId','UserId','CommentCreationDate').withColumn('No_of_comments', f.row_number().over(windowval2_c)), [(df_p.OwnerUserId == df_c.UserId) & (df_c.CommentCreationDate < df_p.CreationDate)],how='left') \
      .select('Id', 'CreationDate', 'CommentCreationDate','No_of_comments') \
      .withColumn('last', f.last('No_of_comments').over(windowval_c)) \
      .filter(f.col('last')==f.col('No_of_comments')) \
      .select('Id','No_of_comments')
        
        
df_questions = df_p.where(df_p.PostTypeId == 1).select('Id','OwnerUserId','Title','Tags','AcceptedAnswerId','CreationDate','LastActivityDate') \
      .withColumn("OwnerUserId_Null", f.when(df_p.OwnerUserId.isNotNull(),1).otherwise(0)) \
      .withColumn("PrevCreationDate", f.lag(df_p.CreationDate).over(windowval2)) \
      .withColumn('No_of_questions_last_year', f.count('Id').over(windowval_q)) \
      .withColumn('No_of_questions', f.row_number().over(windowval2_q)) \
      .join(df_u, [(df_p.OwnerUserId == df_u.OwnerUserId)],how='left') \
      .select("Id","UserCreationDate","Reputation","UserLocation","Flag_Location","Flag_AboutMe","Flag_WebsiteUrl","CreationDate","PrevCreationDate","OwnerUserId_Null","No_of_questions_last_year","No_of_questions") \
      .withColumn("Seniority", f.datediff('CreationDate','UserCreationDate')/365) \
      .withColumn("Time_from_last_question", f.datediff('CreationDate','PrevCreationDate')/365) \
      .withColumn("UserCountry", extractCountry_udf('UserLocation')) \
      .withColumn("UserAlpha2", extractAlpha2_udf('UserCountry')) \
      .withColumn("UserContinent", extractContinent_udf('UserCountry')) \
      .select('Id','Reputation','UserLocation','UserCountry','UserAlpha2','UserContinent','Flag_Location','Flag_AboutMe','Flag_WebsiteUrl','Seniority','OwnerUserId_Null','Time_from_last_question','No_of_questions_last_year','No_of_questions')        
        
#.withColumn("UserContinent", extractContinent_udf('UserCountry')) \        
df_questions.select('Id','UserLocation','UserCountry','UserAlpha2','UserContinent').where(df_questions.UserCountry =='Antarctica').show(1000)        

df_p.where(df_p.PostTypeId == 1) \
    .withColumn("OwnerUserId_Null", f.when(df_p.OwnerUserId.isNotNull(),1).otherwise(0)) \
    .groupby("OwnerUserId_Null") \
    .count().show()
    
    
df_questions.groupby("UserLocation","UserCountry").count().sort(f.col("count").desc()).show(700, truncate = False )    
    

+---+------------+-----------+----------+-------------+
| Id|UserLocation|UserCountry|UserAlpha2|UserContinent|
+---+------------+-----------+----------+-------------+
+---+------------+-----------+----------+-------------+

+----------------+-----+
|OwnerUserId_Null|count|
+----------------+-----+
|               1|54088|
|               0| 1017|
+----------------+-----+

+-------------------------------------------------------------+-------------------------------------------------------------+-----+
|UserLocation                                                 |UserCountry                                                  |count|
+-------------------------------------------------------------+-------------------------------------------------------------+-----+
|null                                                         |Brak                                                         |27420|
|Germany                                                      |Germany                          

In [5]:
from pyspark.sql import Window
from pyspark.sql.types import TimestampType
import pyspark.sql.functions as f

 
           




windowval_b = (Window.partitionBy('UserId').orderBy('BadgeDate')
             .rangeBetween(Window.unboundedPreceding, 0))

windowval2_b = Window.partitionBy("UserId").orderBy("BadgeDate")


#df_p.where(df_p.Id ==3038).select('Id','OwnerUserId','CreationDate').show(truncate = False)
            
df_u_b = df_u.select('OwnerUserId').join(df_b.select('UserId','Name','BadgeDate','Class') 
                  .withColumn('class_3', f.when((df_b.Class == 3), 1).otherwise(0)) \
                  .withColumn('class_2', f.when((df_b.Class == 2), 1).otherwise(0)) \
                  .withColumn('class_1', f.when((df_b.Class == 1), 1).otherwise(0)) \
                  .withColumn("BadgePrevDate", f.lag(df_b.BadgeDate).over(windowval2_b)) \
                  .withColumn('class3', f.sum('Class_3').over(windowval_b)) \
                  .withColumn('class2', f.sum('Class_2').over(windowval_b)) \
                  .withColumn('class1', f.sum('Class_1').over(windowval_b)) , [(df_u.OwnerUserId == df_b.UserId)],how='left')
    
# df_p.where(df_p.OwnerUserId ==1806).select('Id','OwnerUserId','CreationDate') \
#     .join(df_u_b_temp, [(df_p.OwnerUserId == df_u_b_temp.OwnerUserId), (df_p.CreationDate.between(df_u_b_temp.BadgePrevDate,df_u_b_temp.BadgeDate))],how='left') \
#     .select('Id','Name','class3','class2','class1') \
#     .show(truncate = False)            


                
windowval_b = Window.partitionBy("Id").orderBy(f.desc("BadgeDate"))
             
                
                
df_p_u_b = df_p.where(df_p.PostTypeId == 1).select('Id','OwnerUserId','CreationDate') \
    .join(df_u_b,[(df_p.OwnerUserId == df_u_b.OwnerUserId) & (df_p.CreationDate > df_u_b.BadgeDate)],how='inner') \
    .withColumn('rn', f.row_number().over(windowval_b)) \
    .filter("rn==1") \
    .select('Id','class3','class2','class1')  
df_p_u_b.count()    

34971

In [6]:
#5. Wyliczenie
#Ilosci typów glosowań w odniesieniu do daty CreationDate
from pyspark.sql import Window
from pyspark.sql.types import TimestampType
import pyspark.sql.functions as f


windowval = (Window.partitionBy('OwnerUserId').orderBy('VoteId','VoteCreationDate')
             .rangeBetween(Window.unboundedPreceding, 0))




df_p_v = df_p.where(df_p.PostTypeId == 1).select('Id','OwnerUserId','AcceptedAnswerId','CreationDate') \
    .join(df_v.select('VoteId','PostId','VoteTypeId','VoteCreationDate') \
              .withColumn('Up', f.when((df_v.VoteTypeId == 2), 1).otherwise(0)) \
              .withColumn('Down', f.when((df_v.VoteTypeId == 3), 1).otherwise(0)) \
              .withColumn('Fav', f.when((df_v.VoteTypeId == 5), 1).otherwise(0)) \
              .withColumn('Off', f.when((df_v.VoteTypeId == 4), 1).otherwise(0)) \
              .withColumn('Spam', f.when((df_v.VoteTypeId == 12), 1).otherwise(0)) \
              .withColumn('ReOpen', f.when((df_v.VoteTypeId == 7), 1).otherwise(0)) \
              .withColumn('BountyS', f.when((df_v.VoteTypeId == 8), 1).otherwise(0)) \
              .withColumn('BountyC', f.when((df_v.VoteTypeId == 9), 1).otherwise(0)) \
              .withColumn('Close', f.when((df_v.VoteTypeId == 6), 1).otherwise(0)), [(df_p.Id == df_v.PostId) ],how='inner') \
              .withColumn('Up', f.sum('Up').over(windowval)) \
              .withColumn('Down', f.sum('Down').over(windowval)) \
              .withColumn('Fav', f.sum('Fav').over(windowval)) \
              .withColumn('Off', f.sum('Off').over(windowval)) \
              .withColumn('Spam', f.sum('Spam').over(windowval)) \
              .withColumn('ReOpen', f.sum('ReOpen').over(windowval)) \
              .withColumn('BountyS', f.sum('BountyS').over(windowval)) \
              .withColumn('BountyC', f.sum('BountyC').over(windowval)) \
              .withColumn('Close', f.sum('Close').over(windowval)) \
              .select('OwnerUserId','VoteCreationDate','Up','Down','Fav','Close','Off','Spam','ReOpen','BountyS','BountyC') 
                
                
windowval_v = Window.partitionBy("Id").orderBy(f.desc("VoteCreationDate"))
             
                
                
df_p_p_v = df_p.where(df_p.PostTypeId == 1).select('Id','OwnerUserId','CreationDate') \
    .join(df_p_v,[(df_p.OwnerUserId == df_p_v.OwnerUserId) & (df_p.CreationDate > df_p_v.VoteCreationDate)],how='inner') \
    .withColumn('rn', f.row_number().over(windowval_v)) \
    .filter("rn==1") \
    .select('Id','Up','Down','Fav','Close','Off','Spam','BountyS','BountyC','ReOpen') 

df_p_p_v.count()    

49543

In [7]:
import pyspark.sql.functions as f
from pyspark.sql.types import TimestampType
import re
import nltk
import string
from nltk import sent_tokenize,pos_tag,WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from datetime import datetime

replacements = [
    (r'&lt;code&gt;(.*?)&lt;/code&gt;', ' '),
    (r'&lt;a href&quot;(.*?)&quot;&gt;', ' '),
    (r'&lt;/p&gt;',' '),
    (r'&lt;p&gt;',' '),
    (r'&lt;pre&gt;',' '),
    (r'&lt;/pre&gt;',' '),
    (r'&lt;/a&gt;',' '),
    (r'&#xA;',' '),
    (r'&quot;',' '),
    (r'&lt;strong&gt;',' '),
    (r'&lt;BR&gt;',' '),
    (r'&lt;/BR&gt;',' '),
    (r'&lt;/strong&gt;',' '), 
    (r'&lt;em&gt;',' '),
    (r'&lt;/em&gt;',' '),
    (r'&lt;/li&gt;',' '),
    (r'&lt;/ol&gt;',' '),
    (r'&lt;li&gt;',' '),
    (r'&lt;ol&gt;',' '),
    (r'&lt;h1&gt;', ' '),
    (r'&lt;/s&gt;',' '),
    (r'&lt;/s&gt;',' '),
    (r'&lt;div&gt;',' '),
    (r'&lt;/div&gt;',' '),
    (r'&lt;h3&gt;', ' '),
    (r'&lt;/h3&gt;', ' '),
    (r'&lt;ul&gt;', ' '),
    (r'&lt;/ul&gt;', ' '),
    (r'&lt;br/&gt;', ' '),
    (r'#xD;', ' '),
    (r'&lt;hr&gt;', ' '),
    (r'&lt;/hr&gt;', ' '),
    (r'&lt;blockquote&gt;',' '),
    (r'&lt;/blockquote&gt;',' '),
    (r'&lt;h1&gt;Context', ' '),
    (r'&lt;/h1&gt;', ' '),
    (r'&lt;h2&gt;', ' '),
    (r'&lt;h2&gt;Context', ' '),
    (r'&lt;/h2&gt;', ' '),
    (r'@(\w+)',' '),
    (r'\d+',' '),
    (r'\s+', ' ') 
    
    
]



def replace_signs(text):
    return re.sub(r'\&gt\;\&lt\;', ' ',text)

def clear_body(text):
    for old, new in replacements:
        text = re.sub(old, new, text)
    text = text.translate(str.maketrans('','',string.punctuation)) 
    text = text.lower()
    return text    



# def replace_in_body(text):
#     re_code = re.compile('&lt;code&gt;(.*?)&lt;/code&gt;')
#     re_url = re.compile('')

def first_tag(text):
    return re.sub(r'\&gt\;\&lt\;', ' ',text).split(' ',1)[0][1:]

def check_codesnippet(text):
    return '&lt;pre&gt;&lt;code&gt;' in text
def check_codeurl(text):
    return '&lt;a href&quot;' in text

def get_codesnippet(text):
    rc = re.search(r'&lt;code&gt;(.*?)&lt;/code&gt;', text,flags=re.S)
    if rc:
        result = rc.group(1) 
    else:
        result = None
    return result
                     
def check_questionmark(text):
    return '?' in text

def check_language(text):
    if '&lt;c++&gt;' in  text:
        lng = 'c++'
    elif '&lt;python&gt;' in  text:  
        lng = 'python'
    elif '&lt;f#&gt;' in  text:  
        lng = 'f#'
    elif '&lt;c#&gt;' in  text:  
        lng = 'c#'
    elif '&lt;c&gt;' in  text:  
        lng = 'c'
    elif '&lt;objective-c&gt;' in  text:  
        lng = 'c++'
    elif '&lt;bash&gt;' in  text:  
        lng = 'unix'
    elif '&lt;vb.net&gt;' in  text:  
        lng = 'vb.net'
    elif '&lt;vbscript&gt;' in  text:  
        lng = 'vb.net'
    elif '&lt;vb6&gt;' in  text:  
        lng = 'vb.net'
    elif '&lt;vba&gt;' in  text:  
        lng = 'vb.net'
    elif '&lt;clojure&gt;' in  text:  
        lng = 'clojure'
    elif '&lt;go&gt;' in  text:  
        lng = 'go'
    elif '&lt;r&gt;' in  text:  
        lng = 'R'
    elif '&lt;erlang&gt;' in  text:  
        lng = 'R'
    elif '&lt;r&gt;' in  text:  
        lng = 'R'
    elif '&lt;lisp&gt;' in  text:  
        lng = 'lisp'
    elif '&lt;matlab&gt;' in  text:  
        lng = 'matlab'
    elif '&lt;ruby&gt;' in  text:  
        lng = 'ruby'
    elif '&lt;ruby-on-rails&gt;' in  text:  
        lng = 'ruby'
    elif '&lt;lua&gt;' in  text:  
        lng = 'lua'
    elif '&lt;elisp&gt;' in  text:  
        lng = 'elisp'
    elif '&lt;java&gt;' in  text:  
        lng = 'java'
    elif '&lt;scala&gt;' in  text:  
        lng = 'scala'
    elif '&lt;perl&gt;' in  text:  
        lng = 'perl'
    elif '&lt;perl6&gt;' in  text:  
        lng = 'perl'
    elif '&lt;javascript&gt;' in  text:  
        lng = 'javascript'
    elif '.js&gt;' in  text:  
        lng = 'javascript'
    elif '&lt;coffeescript&gt;' in  text:  
        lng = 'javascript'
    elif '&lt;typescript&gt;' in  text:  
        lng = 'javascript'
    elif '&lt;jsx&gt;' in  text:  
        lng = 'javascript'
    elif '&lt;php&gt;' in  text:  
        lng = 'php'
    elif '&lt;php5&gt;' in  text:  
        lng = 'php'
    elif '&lt;haskell&gt;' in  text:  
        lng = 'haskell'
    elif '&lt;rust&gt;' in  text:  
        lng = 'rust'
    elif '&lt;groovy&gt;' in  text:  
        lng = 'groovy'
    elif '&lt;unix&gt;' in  text:  
        lng = 'unix'
    elif '&lt;linux&gt;' in  text:  
        lng = 'unix'
    elif '&lt;html&gt;' in  text:  
        lng = 'html'
    elif '&lt;css&gt;' in  text:  
        lng = 'html'
    elif '&lt;sql&gt;' in  text:  
        lng = 'sql'
    elif '&lt;mysql&gt;' in  text:  
        lng = 'sql'
    elif '&lt;object-pascal&gt;' in  text:  
        lng = 'delphi'
    elif '&lt;pascal&gt;' in  text:  
        lng = 'delphi'
    elif '&lt;delphi&gt;' in  text:  
        lng = 'delphi'
    elif '&lt;swift&gt;' in  text:  
        lng = 'swift'
    elif '&lt;rx-swift&gt;' in  text:  
        lng = 'swift'
    elif '&lt;kotlin&gt;' in  text:  
        lng = 'kotlin' 
    else:
        lng = 'other'
    return lng    
        
def get_number_sentences(text):
    sentences = nltk.sent_tokenize(text)
    return len(sentences)

def remove_tags(text):
    # noun tags
    nn_tags = ['NN', 'NNP', 'NNP', 'NNPS', 'NNS']
    # adjectives
    jj_tags = ['JJ', 'JJR', 'JJS']
    # verbs
    vb_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    nltk_tags = nn_tags + jj_tags + vb_tags
    text1 = nltk.word_tokenize(text)
    tagged_text = pos_tag(text1)
    cleaned_text = [tagged_word[0] for tagged_word in tagged_text if tagged_word[1] in nltk_tags]
    return ' '.join(cleaned_text)

def lemmatize(text):
# expects a string
    text1 = nltk.word_tokenize(text)
    WNLema = WordNetLemmatizer()
    #porter = PorterStemmer()
    #lemmatizer = [porter.stem(w) for w in text1]
    lemmatized = [WNLema.lemmatize(w) for w in text1]
    unique_lemmatized = set(lemmatized) 
    return ' '.join(unique_lemmatized)



stops_w = stopwords.words('english')

def get_number_words(text):
    words = [w  for w in nltk.word_tokenize(text) if w not in stops_w]
    return len(words) 

def get_number_signs(text):
    return len(text)

def get_number_digits(text):
    digits = [s for s in text if s.isdigit()]
    return len(digits)

replaceSignTags_udf = f.udf(lambda x: replace_signs(x))
clearBody_udf = f.udf(lambda x: clear_body(x))
clearBodyTags_udf = f.udf(lambda x: remove_tags(x))
clearBodyLemmatize_udf = f.udf(lambda x: lemmatize(x))
codeSnippet_udf = f.udf(lambda x: check_codesnippet(x)) 
url_udf = f.udf(lambda x: check_codeurl(x)) 
questionMark_udf = f.udf(lambda x: check_questionmark(x))
language_udf = f.udf(lambda x: check_language(x))
first_tag_udf = f.udf(lambda x: first_tag(x))
numberSentences_udf = f.udf(lambda x: get_number_sentences(x))
numberWords_udf = f.udf(lambda x: get_number_words(x))
numberSigns_udf = f.udf(lambda x: get_number_signs(x))
numberDigits_udf = f.udf(lambda x: get_number_digits(x))
extractCodeSnippet_udf = f.udf(lambda x: get_codesnippet(x))
#toDate_udf =  f.udf(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%f'))
#toDate_udf =  f.udf(lambda x: f.to_timestamp(x, 'yyyy-MM-ddTHH:mm:ss.SSS'))


#test = df_p.where((df_p.PostTypeId == '1') & (df_p.OwnerUserId == 71)).select('Id','OwnerUserId','Title','Body','Tags','AcceptedAnswerId','CreationDate')
#test = df_p_q.select('Id','OwnerUserId','Title','Body','Tags','AcceptedAnswerId','CreationDate')
#test.show()
master = df_p.where(df_p.PostTypeId == 1).select('Id','OwnerUserId','Title','Body','Tags','AcceptedAnswerId','CreationDate') \
             .withColumn("word_count_title", f.size(f.split(df_p['Title'], ' '))) \
             .withColumn("word_count_body", f.size(f.split(df_p['Body'], ' '))) \
             .withColumn("no_of_tags", f.size(f.split(replaceSignTags_udf(df_p['Tags']),' '))) \
             .withColumn("flag_bodycodesnippet", f.when(codeSnippet_udf(df_p['Body'])==True,1).otherwise(0)) \
             .withColumn("flag_bodyurl", f.when(url_udf(df_p['Body'])==True,1).otherwise(0)) \
             .withColumn("flag_questionmark_title", f.when(questionMark_udf(df_p['Title'])==True,1).otherwise(0)) \
             .withColumn("flag_questionmark_body", f.when(questionMark_udf(df_p['Body'])==True,1).otherwise(0)) \
             .withColumn("language", language_udf(df_p['Tags'])) \
             .withColumn("first_tag", first_tag_udf(df_p['Tags'])) \
             .withColumn("no_sentences_body", numberSentences_udf(df_p['Body'])) \
             .withColumn("no_words_body", numberWords_udf(df_p['Body'])) \
             .withColumn("no_signs_body", numberSigns_udf(df_p['Body'])) \
             .withColumn("no_digits_body", numberDigits_udf(df_p['Body'])) \
             .withColumn("codesnippet", extractCodeSnippet_udf(df_p['Body'])) \
             .withColumn("body_cleaned", clearBody_udf(df_p['Body'])) \
             .withColumn("body_cleaned2", clearBodyLemmatize_udf(clearBodyTags_udf(clearBody_udf(df_p['Body'])))) \
             .withColumn("Target", f.when(df_p.AcceptedAnswerId.isNotNull(),1).otherwise(0)) 
#master.show(truncate = False)
#master.select("Id","Body","Tags","language","if_codesnippet","body_without_codesnippet").show(truncate = False)
master.select("Id","body","body_cleaned","body_cleaned2","flag_bodyurl").show(3,truncate = False)

# .withColumn("no_words_body", numberWords_udf(df_p['Body'])) \
#              .withColumn("no_signs_body", numberSigns_udf(df_p['Body'])) \
#  .withColumn("no_signs_codesnippet", numberSigns_udf(extractCodeSnippet_udf(df_p['Body']))) \ 
#              .withColumn("no_digits_codesnippet", numberDigits_udf(extractCodeSnippet_udf(df_p['Body']))) \ 

+---+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
from pyspark.sql import Window
from pyspark.sql.types import TimestampType,LongType
import pyspark.sql.functions as f

#test = df_p_q.where((df_p_q.OwnerUserId == 13452)).select('Id','OwnerUserId','Title','Body','Tags','AcceptedAnswerId','CreationDate')
#test.show()
#test2 = df_p_a.where((df_p_a.OwnerUserId == 13452)).select('Id','ParentId','OwnerUserId','Title','Body','Tags','AcceptedAnswerId','AnswerCreationDate')
#test2.show()



windowval2_a = Window.partitionBy("OwnerUserId").orderBy("AnswerCreationDate")

windowval = Window.partitionBy("Id").orderBy("CreationDate")


df_p_q_a_no  = df_p.where(df_p.PostTypeId == 1).select('Id','OwnerUserId','CreationDate') \
      .join(df_p_a.select('OwnerUserId','AnswerCreationDate').withColumn('No_of_answers', f.row_number().over(windowval2_a)), [(df_p.OwnerUserId == df_p_a.OwnerUserId) & (df_p_a.AnswerCreationDate < df_p.CreationDate)],how='inner') \
      .withColumn('last', f.last('No_of_answers').over(windowval)) \
      .filter(f.col('last')==f.col('No_of_answers')) \
      .select('Id','No_of_answers')
df_p_q_a_no.count()        

10114

In [9]:
df_p_q_c = df_p.where(df_p.PostTypeId == 1).join(df_c.select('CommentId','UserId','CommentCreationDate').withColumn('No_of_comments', f.row_number().over(windowval2_c)), [(df_p.OwnerUserId == df_c.UserId) & (df_c.CommentCreationDate < df_p.CreationDate)],how='left') \
      .select('Id', 'CreationDate', 'CommentCreationDate','No_of_comments') \
      .withColumn('last', f.last('No_of_comments').over(windowval_c)) \
      .filter(f.col('last')==f.col('No_of_comments')) \
      .select('Id','No_of_comments')
df_p_q_c.count()        

24586

In [10]:

        
#49543 df_p_p_v
#10114 df_p_q_a_no
#34971 df_p_u_b        
#24586 df_p_q_c        


df_final = df_questions.join(master, ["Id"],how='inner') \
            .join(df_p_p_v, ["Id"],how='left') \
            .join(df_p_u_b, ["Id"],how='left') \
            .join(df_p_q_a_no, ["Id"],how='left') \
            .join(df_p_q_c, ["Id"],how='left') \
            .withColumn("No_of_answers",f.when(df_p_q_a_no.No_of_answers.isNotNull(),df_p_q_a_no.No_of_answers).otherwise(0)) \
            .withColumn("No_of_comments",f.when(df_p_q_c.No_of_comments.isNotNull(),df_p_q_c.No_of_comments).otherwise(0)) \
            .withColumn("Up",f.when(df_p_p_v.Up.isNotNull(),df_p_p_v.Up).otherwise(0)) \
            .withColumn("Down",f.when(df_p_p_v.Down.isNotNull(),df_p_p_v.Down).otherwise(0)) \
            .withColumn("Fav",f.when(df_p_p_v.Fav.isNotNull(),df_p_p_v.Fav).otherwise(0)) \
            .withColumn("Off",f.when(df_p_p_v.Off.isNotNull(),df_p_p_v.Off).otherwise(0)) \
            .withColumn("Spam",f.when(df_p_p_v.Spam.isNotNull(),df_p_p_v.Spam).otherwise(0)) \
            .withColumn("ReOpen",f.when(df_p_p_v.ReOpen.isNotNull(),df_p_p_v.ReOpen).otherwise(0)) \
            .withColumn("BountyC",f.when(df_p_p_v.BountyC.isNotNull(),df_p_p_v.BountyC).otherwise(0)) \
            .withColumn("BountyS",f.when(df_p_p_v.BountyS.isNotNull(),df_p_p_v.BountyS).otherwise(0)) \
            .withColumn("Close",f.when(df_p_p_v.Close.isNotNull(),df_p_p_v.Close).otherwise(0)) \
            .withColumn("class1",f.when(df_p_u_b.class1.isNotNull(),df_p_u_b.class1).otherwise(0)) \
            .withColumn("class2",f.when(df_p_u_b.class2.isNotNull(),df_p_u_b.class2).otherwise(0)) \
            .withColumn("class3",f.when(df_p_u_b.class3.isNotNull(),df_p_u_b.class3).otherwise(0)) 

# df_questions.join(master, ["Id"],how='inner') \
#             .join(df_p_p_v, ["Id"],how='left') \
#             .join(df_p_u_b, ["Id"],how='left') \
#             .join(df_p_q_a_no, ["Id"],how='left') \
#             .join(df_p_q_c, ["Id"],how='left') \
#             .withColumn("No_of_answers",f.when(df_p_q_a_no.No_of_answers.isNotNull(),df_p_q_a_no.No_of_answers).otherwise(0)) \
#             .withColumn("No_of_comments",f.when(df_p_q_c.No_of_comments.isNotNull(),df_p_q_c.No_of_comments).otherwise(0)) \
#             .withColumn("Up",f.when(df_p_p_v.Up.isNotNull(),df_p_p_v.Up).otherwise(0)) \
#             .withColumn("Down",f.when(df_p_p_v.Down.isNotNull(),df_p_p_v.Down).otherwise(0)) \
#             .withColumn("Fav",f.when(df_p_p_v.Fav.isNotNull(),df_p_p_v.Fav).otherwise(0)) \
#             .withColumn("Close",f.when(df_p_p_v.Close.isNotNull(),df_p_p_v.Close).otherwise(0)) \
#             .withColumn("class1",f.when(df_p_u_b.class1.isNotNull(),df_p_u_b.class1).otherwise(0)) \
#             .withColumn("class2",f.when(df_p_u_b.class2.isNotNull(),df_p_u_b.class2).otherwise(0)) \
#             .withColumn("class3",f.when(df_p_u_b.class3.isNotNull(),df_p_u_b.class3).otherwise(0)) \
#             .show(truncate = False)            
            

# df_questions.where(df_questions.Id==101844).join(master, ["Id"],how='inner') \
#             .join(df_p_p_v, ["Id"],how='left') \
#             .join(df_p_u_b, ["Id"],how='left') \
#             .join(df_p_q_a_no, ["Id"],how='left') \
#             .join(df_p_q_c, ["Id"],how='left') \
#             .withColumn("No_of_answers",f.when(df_p_q_a_no.No_of_answers.isNotNull(),df_p_q_a_no.No_of_answers).otherwise(0)) \
#             .withColumn("No_of_comments",f.when(df_p_q_c.No_of_comments.isNotNull(),df_p_q_c.No_of_comments).otherwise(0)) \
#             .withColumn("Up",f.when(df_p_p_v.Up.isNotNull(),df_p_p_v.Up).otherwise(0)) \
#             .withColumn("Down",f.when(df_p_p_v.Down.isNotNull(),df_p_p_v.Down).otherwise(0)) \
#             .withColumn("Fav",f.when(df_p_p_v.Fav.isNotNull(),df_p_p_v.Fav).otherwise(0)) \
#             .withColumn("Close",f.when(df_p_p_v.Close.isNotNull(),df_p_p_v.Close).otherwise(0)) \
#             .withColumn("class1",f.when(df_p_u_b.class1.isNotNull(),df_p_u_b.class1).otherwise(0)) \
#             .withColumn("class2",f.when(df_p_u_b.class2.isNotNull(),df_p_u_b.class2).otherwise(0)) \
#             .withColumn("class3",f.when(df_p_u_b.class3.isNotNull(),df_p_u_b.class3).otherwise(0)) \
#             .show(truncate = False)
        
# .withColumn("No_of_answers",f.when(df_p_q_a_no.No_of_answers.isNotNull(),1).otherwise(0)).

In [11]:
df_final.write.parquet("/home/adam/bigdata/dane/codereview/stack_table.parquet", mode='overwrite')

In [12]:
import pandas as pd

table = pd.read_parquet("/home/adam/bigdata/dane/codereview/stack_table.parquet")

table.info()
table.head()
#table2 = pq.read_table("/home/adam/bigdata/dane/codereview/stack_table.parquet").to_pandas()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55105 entries, 0 to 55104
Data columns (total 51 columns):
Id                           55105 non-null int32
Reputation                   54088 non-null float64
UserLocation                 27685 non-null object
UserCountry                  55105 non-null object
UserAlpha2                   23624 non-null object
UserContinent                55105 non-null object
Flag_Location                54088 non-null float64
Flag_AboutMe                 54088 non-null float64
Flag_WebsiteUrl              54088 non-null float64
Seniority                    54088 non-null float64
OwnerUserId_Null             55105 non-null int32
Time_from_last_question      27234 non-null float64
No_of_questions_last_year    55105 non-null int64
No_of_questions              55105 non-null int32
OwnerUserId                  54088 non-null float64
Title                        55105 non-null object
Body                         55105 non-null object
Tags                 

,Id,Reputation,UserLocation,UserCountry,UserAlpha2,UserContinent,Flag_Location,Flag_AboutMe,Flag_WebsiteUrl,Seniority,...,Off,Spam,BountyS,BountyC,ReOpen,class3,class2,class1,No_of_answers,No_of_comments
0,326,363.0,"La Crosse, WI",United States of America,US,NA,1.0,0.0,1.0,0.019178,...,0,0,0,0,0,1,0,0,0,0
1,471,254.0,"Amsterdam, MO",United States of America,US,NA,1.0,1.0,1.0,0.000000,...,0,0,0,0,0,1,0,0,0,0
2,1959,153.0,"Buenos Aires, Argentina",Argentina,AR,SA,1.0,1.0,0.0,0.032877,...,0,0,0,0,0,1,0,0,0,0
3,3273,170.0,"Windsor, Ontario, Canada",Canada,CA,NA,1.0,1.0,1.0,0.090411,...,0,0,0,0,0,1,0,0,0,1
4,4101,123.0,South Africa,South Africa,ZA,AF,1.0,1.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [1]:
from pyspark.sql import Window
from pyspark.sql.types import TimestampType,LongType
import pyspark.sql.functions as f

#test = df_p_q.where((df_p_q.OwnerUserId == 13452)).select('Id','OwnerUserId','Title','Body','Tags','AcceptedAnswerId','CreationDate')
#test.show()
#test2 = df_p_a.where((df_p_a.OwnerUserId == 13452)).select('Id','ParentId','OwnerUserId','Title','Body','Tags','AcceptedAnswerId','AnswerCreationDate')
#test2.show()



windowval2_a = Window.partitionBy("OwnerUserId").orderBy("AnswerCreationDate")

windowval = Window.partitionBy("Id").orderBy("CreationDate")


df_p_q_a_no  = df_p.where((df_p.PostTypeId == 1) & (df_p.Id == 101844)).select('Id','OwnerUserId','CreationDate') \
      .join(df_p_a.select('OwnerUserId','AnswerCreationDate').withColumn('No_of_answers', f.row_number().over(windowval2_a)), [(df_p.OwnerUserId == df_p_a.OwnerUserId) & (df_p_a.AnswerCreationDate < df_p.CreationDate)],how='left') \
      .withColumn('last', f.last('No_of_answers').over(windowval)) 
df_p_q_a_no.show() 


df_p_q_c = df_p.where((df_p.PostTypeId == 1)  & (df_p.Id == 101844)).join(df_c.select('CommentId','UserId','CommentCreationDate').withColumn('No_of_comments', f.row_number().over(windowval2_c)), [(df_p.OwnerUserId == df_c.UserId) & (df_c.CommentCreationDate < df_p.CreationDate)],how='inner') \
      .select('Id', 'CreationDate', 'CommentCreationDate','No_of_comments') \
      .withColumn('last', f.last('No_of_comments').over(windowval_c)) 
      
df_p_q_c.show()  


from pyspark.sql import Window
from pyspark.sql.types import TimestampType
import pyspark.sql.functions as f


windowval = (Window.partitionBy('OwnerUserId').orderBy('VoteId','VoteCreationDate')
             .rangeBetween(Window.unboundedPreceding, 0))




df_p_v = df_p.where((df_p.PostTypeId == 1) & (df_p.Id == 101844)).select('Id','OwnerUserId','AcceptedAnswerId','CreationDate') \
    .join(df_v.select('VoteId','PostId','VoteTypeId','VoteCreationDate') \
              .withColumn('Up', f.when((df_v.VoteTypeId == 2), 1).otherwise(0)) \
              .withColumn('Down', f.when((df_v.VoteTypeId == 3), 1).otherwise(0)) \
              .withColumn('Fav', f.when((df_v.VoteTypeId == 5), 1).otherwise(0)) \
              .withColumn('Close', f.when((df_v.VoteTypeId == 6), 1).otherwise(0)), [(df_p.Id == df_v.PostId) ],how='inner') \
              .withColumn('Up', f.sum('Up').over(windowval)) \
              .withColumn('Down', f.sum('Down').over(windowval)) \
              .withColumn('Fav', f.sum('Fav').over(windowval)) \
              .withColumn('Close', f.sum('Close').over(windowval)) \
              .select('OwnerUserId','VoteCreationDate','Up','Down','Fav','Close') 
                
                
windowval_v = Window.partitionBy("Id").orderBy(f.desc("VoteCreationDate"))
             
                
                
df_p_p_v = df_p.where((df_p.PostTypeId == 1) & (df_p.Id == 101844)) .select('Id','OwnerUserId','CreationDate') \
    .join(df_p_v,[(df_p.OwnerUserId == df_p_v.OwnerUserId) & (df_p.CreationDate > df_p_v.VoteCreationDate)],how='inner') \
    .withColumn('rn', f.row_number().over(windowval_v)) \
    .select('Id','Up','Down','Fav','Close','CreationDate','VoteCreationDate','rn') 

df_p_p_v.show()   


from pyspark.sql import Window
from pyspark.sql.types import TimestampType
import pyspark.sql.functions as f

 
           




windowval_b = (Window.partitionBy('UserId').orderBy('BadgeDate')
             .rangeBetween(Window.unboundedPreceding, 0))

windowval2_b = Window.partitionBy("UserId").orderBy("BadgeDate")


#df_p.where(df_p.Id ==3038).select('Id','OwnerUserId','CreationDate').show(truncate = False)
            
df_u_b = df_u.select('OwnerUserId').join(df_b.select('UserId','Name','BadgeDate','Class') 
                  .withColumn('class_3', f.when((df_b.Class == 3), 1).otherwise(0)) \
                  .withColumn('class_2', f.when((df_b.Class == 2), 1).otherwise(0)) \
                  .withColumn('class_1', f.when((df_b.Class == 1), 1).otherwise(0)) \
                  .withColumn("BadgePrevDate", f.lag(df_b.BadgeDate).over(windowval2_b)) \
                  .withColumn('class3', f.sum('Class_3').over(windowval_b)) \
                  .withColumn('class2', f.sum('Class_2').over(windowval_b)) \
                  .withColumn('class1', f.sum('Class_1').over(windowval_b)) , [(df_u.OwnerUserId == df_b.UserId)],how='left')
    
# df_p.where(df_p.OwnerUserId ==1806).select('Id','OwnerUserId','CreationDate') \
#     .join(df_u_b_temp, [(df_p.OwnerUserId == df_u_b_temp.OwnerUserId), (df_p.CreationDate.between(df_u_b_temp.BadgePrevDate,df_u_b_temp.BadgeDate))],how='left') \
#     .select('Id','Name','class3','class2','class1') \
#     .show(truncate = False)            


                
windowval_b = Window.partitionBy("Id").orderBy(f.desc("BadgeDate"))
             
                
                
df_p_u_b = df_p.where((df_p.PostTypeId == 1) & (df_p.Id == 101844)).select('Id','OwnerUserId','CreationDate') \
    .join(df_u_b,[(df_p.OwnerUserId == df_u_b.OwnerUserId) & (df_p.CreationDate > df_u_b.BadgeDate)],how='inner') \
    .withColumn('rn', f.row_number().over(windowval_b)) \
    .select('Id','class3','class2','class1','CreationDate','BadgeDate','rn')  
df_p_u_b.show()    


AttributeError: 'NoneType' object has no attribute '_jvm'

In [4]:
def transform_element(RDD_elem, elem_struct, names, dt_fmt):
    out = dict()
    for elem, e_type, name in zip(RDD_elem, elem_struct, names):
        if e_type == "int":
            if elem == "":
                elem = 9999
            else:
                elem = int(elem)
        elif e_type == "datetime":
            elem = datetime.strptime(elem, dt_fmt)
        elif e_type == "float":
            elem = float(elem)
        out[name] = elem
    return out

In [18]:

from datetime import datetime, timedelta
dt_format = "%Y-%m-%dT%H:%M:%S.%f"
x = transData.take(1)
transform_element(x, struct, col_names, dt_format)



TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [20]:
df_badge = transData.map(lambda row: transform_element(row, struct, col_names, dt_format))
df_badge.take(5)

[{'BadgeDate': datetime.datetime(2011, 1, 19, 20, 52, 2, 27000),
  'Class': 3,
  'Id': 1,
  'Name': 'Autobiographer',
  'TagBased': 'False',
  'UserId': 2},
 {'BadgeDate': datetime.datetime(2011, 1, 19, 20, 57, 2, 100000),
  'Class': 3,
  'Id': 2,
  'Name': 'Autobiographer',
  'TagBased': 'False',
  'UserId': 4},
 {'BadgeDate': datetime.datetime(2011, 1, 19, 20, 57, 2, 133000),
  'Class': 3,
  'Id': 3,
  'Name': 'Autobiographer',
  'TagBased': 'False',
  'UserId': 6},
 {'BadgeDate': datetime.datetime(2011, 1, 19, 20, 57, 2, 147000),
  'Class': 3,
  'Id': 4,
  'Name': 'Autobiographer',
  'TagBased': 'False',
  'UserId': 7},
 {'BadgeDate': datetime.datetime(2011, 1, 19, 20, 57, 2, 163000),
  'Class': 3,
  'Id': 5,
  'Name': 'Autobiographer',
  'TagBased': 'False',
  'UserId': 8}]

In [29]:
from pyspark.sql import Row
sc.createDataFrame(Row(**x) for x in df_b).show(truncate=False)

AttributeError: 'SparkContext' object has no attribute 'createDataFrame'

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

#spark.createDataFrame(df_badge).show(truncate=False)
#df_badge.map(lambda l: Row(**dict(l))).toDF().show(truncate=False)
df_b = df_badge.map(lambda l: Row(**dict(l))).toDF()

#df_b = sc.parallelize(df_badge).toDF()
#df_b = spark.createDataFrame(Row(**x) for x in list(df_badge))
#df_b = sc.parallelize(df_badge).map(lambda x: Row(**OrderedDict(sorted(x.items())))).toDF()
#df_b = spark.sparkContext.parallelize(df_badge).map(lambda x: Row(**x)).toDF()

In [27]:
from incf.countryutils import transformations

ModuleNotFoundError: No module named 'incf'

In [28]:
import sys
!conda install --yes --prefix {sys.prefix} incf.countryutils

Solving environment: failed

PackagesNotFoundError: The following packages are not available from current channels:

  - incf.countryutils

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/free/linux-64
  - https://repo.anaconda.com/pkgs/free/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/pro/linux-64
  - https://repo.anaconda.com/pkgs/pro/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [12]:
punc_re = re.compile('[%s]' % re.escape(string.punctuation))
punc_re

NameError: name 'string' is not defined

In [19]:
z = string.punctuation
z

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [30]:
co = list(pycountry.countries)
co

[Country(alpha_2='AW', alpha_3='ABW', name='Aruba', numeric='533'),
 Country(alpha_2='AF', alpha_3='AFG', name='Afghanistan', numeric='004', official_name='Islamic Republic of Afghanistan'),
 Country(alpha_2='AO', alpha_3='AGO', name='Angola', numeric='024', official_name='Republic of Angola'),
 Country(alpha_2='AI', alpha_3='AIA', name='Anguilla', numeric='660'),
 Country(alpha_2='AX', alpha_3='ALA', name='Åland Islands', numeric='248'),
 Country(alpha_2='AL', alpha_3='ALB', name='Albania', numeric='008', official_name='Republic of Albania'),
 Country(alpha_2='AD', alpha_3='AND', name='Andorra', numeric='020', official_name='Principality of Andorra'),
 Country(alpha_2='AE', alpha_3='ARE', name='United Arab Emirates', numeric='784'),
 Country(alpha_2='AR', alpha_3='ARG', name='Argentina', numeric='032', official_name='Argentine Republic'),
 Country(alpha_2='AM', alpha_3='ARM', name='Armenia', numeric='051', official_name='Republic of Armenia'),
 Country(alpha_2='AS', alpha_3='ASM', nam

In [31]:
import sys
!{sys.executable} -m pip install pycountry-convert


    100% |████████████████████████████████| 225kB 2.0MB/s ta 0:00:01
    100% |████████████████████████████████| 215kB 2.5MB/s eta 0:00:01
    100% |████████████████████████████████| 51kB 2.9MB/s eta 0:00:01
    100% |████████████████████████████████| 92kB 3.9MB/s eta 0:00:01
  Found existing installation: py 1.4.34
    Uninstalling py-1.4.34:
      Successfully uninstalled py-1.4.34
  Found existing installation: pytest 3.2.1
    Uninstalling pytest-3.2.1:
      Successfully uninstalled pytest-3.2.1
  Found existing installation: wheel 0.29.0
    Uninstalling wheel-0.29.0:
      Successfully uninstalled wheel-0.29.0
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [46]:
from pycountry.convert import country_name_to_country_alpha2

ModuleNotFoundError: No module named 'pycountry.convert'

In [16]:
from pycountry_convert import (
    map_countries,
    country_alpha2_to_country_name,
    country_name_to_country_alpha2,
    country_alpha3_to_country_alpha2,
    country_alpha2_to_continent_code,
    COUNTRY_NAME_FORMAT_DEFAULT
)
text ='Canada'
print(text.strip())
# z = country_name_to_country_alpha2(text)
# z
# cn_name_format = COUNTRY_NAME_FORMAT_DEFAULT
# for key, value in sorted(map_countries(cn_name_format).items()):
#     print(key)
#     if text in key:
#         zz = country_alpha2_to_continent_code(country_name_to_country_alpha2(text.strip()))
        
# print("\n")        
# print(zz)
country_map = [key for key in sorted(map_countries(cn_name_format = COUNTRY_NAME_FORMAT_DEFAULT).keys())]  
print(country_map)
def get_Continent(text): 
    print(text)
    cn_name_format = COUNTRY_NAME_FORMAT_DEFAULT
    for key, value in sorted(map_countries(cn_name_format).items()):
        if text in key:
            continent = country_alpha2_to_continent_code(country_name_to_country_alpha2(text))
    return continent 
print(get_Continent(text))

Canada
['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Arab Republic of Egypt', 'Argentina', 'Argentine Republic', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivarian Republic of Venezuela', 'Bolivia', 'Bolivia, Plurinational State of', 'Bonaire', 'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', 'British Indian Ocean Territory', 'British Virgin Islands', 'Brunei', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands', 'Colombia', 'Commonwealth of Dominica', 'Commonwealth of the Bahamas', 'Commonwealth of the Northern Mariana Islands', 'Comoros', 'Co

In [59]:
stops = set(stopwords.words("english"))
stops

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [67]:
import nltk
znltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/adam/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [92]:
z = 'basically i had written this class a little while ago to ease autoloading of our local libraries the premise is that everything is split by packages into multiple layers of subpackages classes are named using camelcasing so a class name is related to its package as follows  now each package can have package specific interfaces defined by for interfaces exceptions by  etc i tried to make it flexible enough for reuse its also fully unit tested what are your thoughts is it overcomplex '
from pyspark.sql.functions import concat_ws




def remove_tags(text):
    # noun tags
    nn_tags = ['NN', 'NNP', 'NNP', 'NNPS', 'NNS']
    # adjectives
    jj_tags = ['JJ', 'JJR', 'JJS']
    # verbs
    vb_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    nltk_tags = nn_tags + jj_tags + vb_tags
    text1 = nltk.word_tokenize(text)
    tagged_text = pos_tag(text1)
    cleaned_text = [tagged_word[0] for tagged_word in tagged_text if tagged_word[1] in nltk_tags]
    return ' '.join(cleaned_text)

remove_tags(z)

{' ',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y'}

In [93]:
z = 'i had written class little while ease autoloading local libraries premise is everything is split packages multiple layers subpackages classes are named using camelcasing class name is related package follows package have package specific interfaces defined interfaces exceptions etc i tried make flexible reuse unit tested are thoughts is overcomplex'
def lemmatize(text):
# expects a string
    text1 = nltk.word_tokenize(text)
    WNLema = WordNetLemmatizer()
    #porter = PorterStemmer()
    #lemmatizer = [porter.stem(w) for w in text1]
    lemmatized = [WNLema.lemmatize(w) for w in text1]
    unique_lemmatized = set(lemmatized) 
    return ' '.join(unique_lemmatized)
lemmatize(z)

'class package autoloading follows flexible i everything library using ease interface reuse camelcasing etc defined tested layer had make overcomplex subpackages tried exception have written related little unit split local specific name while multiple are named is thought premise'

In [82]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/adam/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [1]:
!pip freeze

alabaster==0.7.10
anaconda-client==1.6.5
anaconda-navigator==1.6.9
anaconda-project==0.8.0
asn1crypto==0.22.0
astroid==1.5.3
astropy==2.0.2
atomicwrites==1.2.1
attrs==18.2.0
Babel==2.5.0
backports.shutil-get-terminal-size==1.0.0
beautifulsoup4==4.6.0
bitarray==0.8.1
bkcharts==0.2
blaze==0.11.3
bleach==2.0.0
bokeh==0.12.10
boto==2.48.0
Bottleneck==1.2.1
certifi==2018.10.15
cffi==1.10.0
chardet==3.0.4
click==6.7
cloudpickle==0.4.0
clyent==1.2.2
colorama==0.3.9
conda==4.5.11
conda-build==3.0.27
conda-verify==2.0.0
contextlib2==0.5.5
coverage==4.5.2
cryptography==2.0.3
cycler==0.10.0
Cython==0.26.1
cytoolz==0.8.2
dask==0.15.3
datashape==0.5.4
decorator==4.1.2
distributed==1.19.1
docutils==0.14
entrypoints==0.2.3
et-xmlfile==1.0.1
fastcache==1.0.2
filelock==2.0.12
findspark==1.2.0
Flask==0.12.2
Flask-Cors==3.0.3
gevent==1.2.2
glob2==0.5
gmpy2==2.0.8
greenlet==0.4.12
h5py==2.7.0
heapdict==1.0.0
html5lib==0.999999999
idna==2.6
imageio==2.2.0
imagesize==0.7.1
ipykernel==4.6.1
ipython==6.1.0
ip